In [2]:
import pandas as pd
from openai import AzureOpenAI
import json
import openai
import pickle
import sys
import os

In [3]:
sys.path.append(os.path.join(os.getcwd(), '..'))

In [5]:
from prompts import *

In [8]:
employment_ground_truth = pd.read_csv('../eval_employment_prompt.csv')
relation_ground_truth = pd.read_csv('../eval_relationship_prompt.csv')

In [9]:
employment_ground_truth.head()

,index,provider_type,patient_id,note_id,sentence_index,text,TRANSPORTATION_distance,TRANSPORTATION_resource,TRANSPORTATION_other,HOUSING_poor,...,EMPLOYMENT_student,SUPPORT_plus,SUPPORT_minus,PARENT,UNKNOWN,llm_text,llm_label_EMPLOYMENT,llm_label_UNKNOWN,EMPLOYMENT_nonadverse,EMPLOYMENT_adverse
0,0,Nursing,10954,401903,0,Ms. [**Known lastname 11692**] is a 52 year ol...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False
1,1,Nursing,10954,401903,1,"During her admission, she developed obtundatio...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False
2,2,Nursing,10954,401903,2,She was extubated the next day after a thorace...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False
3,3,Nursing,10954,401903,3,She was transferred to the medical service fro...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False
4,4,Nursing,10954,401903,4,"Please see Vascular, SICU and Medicine admissi...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False


In [10]:
relation_ground_truth.head()

,index,provider_type,patient_id,note_id,sentence_index,text,TRANSPORTATION_distance,TRANSPORTATION_resource,TRANSPORTATION_other,HOUSING_poor,...,SUPPORT_plus,SUPPORT_minus,PARENT,UNKNOWN,llm_text,llm_label_marital_status,llm_label_RELATIONSHIP,llm_label_UNKNOWN,RELATIONSHIP_nonadverse,RELATIONSHIP_adverse
0,0,Nursing,10954,401903,0,Ms. [**Known lastname 11692**] is a 52 year ol...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,True,"{\n ""marital_status"": ""UNKNOWN"",\n ""marital_...",UNKNOWN,UNKNOWN,True,False,False
1,1,Nursing,10954,401903,1,"During her admission, she developed obtundatio...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,True,"{\n ""marital_status"": ""UNKNOWN"",\n ""marital_...",UNKNOWN,UNKNOWN,True,False,False
2,2,Nursing,10954,401903,2,She was extubated the next day after a thorace...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,True,"{\n ""marital_status"": ""UNKNOWN"",\n ""marital_...",UNKNOWN,UNKNOWN,True,False,False
3,3,Nursing,10954,401903,3,She was transferred to the medical service fro...,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,True,"{\n ""marital_status"": ""UNKNOWN"",\n ""marital_...",UNKNOWN,UNKNOWN,True,False,False
4,4,Nursing,10954,401903,4,"Please see Vascular, SICU and Medicine admissi...",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,True,"{\n ""marital_status"": ""UNKNOWN"",\n ""marital_...",UNKNOWN,UNKNOWN,True,False,False


In [11]:
employment_ground_truth.columns

Index(['index', 'provider_type', 'patient_id', 'note_id', 'sentence_index',
       'text', 'TRANSPORTATION_distance', 'TRANSPORTATION_resource',
       'TRANSPORTATION_other', 'HOUSING_poor', 'HOUSING_undomiciled',
       'HOUSING_other', 'RELATIONSHIP_married', 'RELATIONSHIP_partnered',
       'RELATIONSHIP_divorced', 'RELATIONSHIP_widowed', 'RELATIONSHIP_single',
       'EMPLOYMENT_employed', 'EMPLOYMENT_underemployed',
       'EMPLOYMENT_unemployed', 'EMPLOYMENT_disability', 'EMPLOYMENT_retired',
       'EMPLOYMENT_student', 'SUPPORT_plus', 'SUPPORT_minus', 'PARENT',
       'UNKNOWN', 'llm_text', 'llm_label_EMPLOYMENT', 'llm_label_UNKNOWN',
       'EMPLOYMENT_nonadverse', 'EMPLOYMENT_adverse'],
      dtype='object')

In [12]:
relation_ground_truth.columns

Index(['index', 'provider_type', 'patient_id', 'note_id', 'sentence_index',
       'text', 'TRANSPORTATION_distance', 'TRANSPORTATION_resource',
       'TRANSPORTATION_other', 'HOUSING_poor', 'HOUSING_undomiciled',
       'HOUSING_other', 'RELATIONSHIP_married', 'RELATIONSHIP_partnered',
       'RELATIONSHIP_divorced', 'RELATIONSHIP_widowed', 'RELATIONSHIP_single',
       'EMPLOYMENT_employed', 'EMPLOYMENT_underemployed',
       'EMPLOYMENT_unemployed', 'EMPLOYMENT_disability', 'EMPLOYMENT_retired',
       'EMPLOYMENT_student', 'SUPPORT_plus', 'SUPPORT_minus', 'PARENT',
       'UNKNOWN', 'llm_text', 'llm_label_marital_status',
       'llm_label_RELATIONSHIP', 'llm_label_UNKNOWN',
       'RELATIONSHIP_nonadverse', 'RELATIONSHIP_adverse'],
      dtype='object')

In [13]:
relation_df = relation_ground_truth[['RELATIONSHIP_nonadverse', 'RELATIONSHIP_adverse']]

In [14]:
final_df = employment_ground_truth.join(relation_df, on='index', how='left')

In [15]:
final_df.head()

,index,provider_type,patient_id,note_id,sentence_index,text,TRANSPORTATION_distance,TRANSPORTATION_resource,TRANSPORTATION_other,HOUSING_poor,...,SUPPORT_minus,PARENT,UNKNOWN,llm_text,llm_label_EMPLOYMENT,llm_label_UNKNOWN,EMPLOYMENT_nonadverse,EMPLOYMENT_adverse,RELATIONSHIP_nonadverse,RELATIONSHIP_adverse
0,0,Nursing,10954,401903,0,Ms. [**Known lastname 11692**] is a 52 year ol...,0.0,0.0,0.0,0.0,...,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False,False,False
1,1,Nursing,10954,401903,1,"During her admission, she developed obtundatio...",0.0,0.0,0.0,0.0,...,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False,False,False
2,2,Nursing,10954,401903,2,She was extubated the next day after a thorace...,0.0,0.0,0.0,0.0,...,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False,False,False
3,3,Nursing,10954,401903,3,She was transferred to the medical service fro...,0.0,0.0,0.0,0.0,...,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False,False,False
4,4,Nursing,10954,401903,4,"Please see Vascular, SICU and Medicine admissi...",0.0,0.0,0.0,0.0,...,0.0,0.0,True,"{\n ""EMPLOYMENT"": ""UNKNOWN"",\n ""EMPLOYMENT_C...",UNKNOWN,True,False,False,False,False


In [17]:
with open('../azure_credentials.json', 'r') as file:
    azure_data = json.load(file)
    api_key = azure_data['API_KEY']
    api_version = azure_data['API_VERSION']
    azure_endpoint = azure_data['AZURE_ENDPOINT']
    azure_deployment_name = azure_data['AZURE_DEPLOYMENT_NAME']

client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint = azure_endpoint
    )

deployment_name=azure_deployment_name #This will correspond to the custom name you chose for your deployment when you deployed a model. Use a gpt-35-turbo-instruct deployment.

# Defining a function to create the prompt from the instruction system message, the few-shot examples, and the current query
def create_prompt(system_message, user_message):    
    formatted_message = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]
    
    return formatted_message

# This function sends the prompt to the GPT model
def send_message(message, model_name, max_response_tokens=500):
    response = client.chat.completions.create(
        model=model_name,
        messages=message,
        temperature=0,
        max_tokens=max_response_tokens,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    
    return response.choices[0].message.content.strip()

In [18]:
final_df['EMPLOYMENT_nonadverse'].value_counts()

EMPLOYMENT_nonadverse
False    5276
True       45
Name: count, dtype: int64

In [19]:
# index_list = []
# llm_response_list = []
# system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
# for index, row in final_df[final_df['EMPLOYMENT_nonadverse'] == True].iterrows():
#     free_text = row['text']
#     user_message = step1_query_optimized.format(free_text=free_text)
#     openai_message = create_prompt(system_message, user_message)
#     response = send_message(openai_message, deployment_name)
    
#     index_list.append(index)
#     llm_response_list.append(response)
#     print(free_text)
#     print(response)
#     print()

# llm_employment_nonadverse_step1 = pd.DataFrame({'index': index_list, 'llm_employment_nonadverse': llm_response_list})

# with open('llm_employment_nonadverse_step1.pkl', 'wb') as file:
#     pickle.dump(llm_employment_nonadverse_step1, file)

In [20]:
final_df['EMPLOYMENT_adverse'].value_counts()

EMPLOYMENT_adverse
False    5299
True       22
Name: count, dtype: int64

In [21]:
# index_list = []
# llm_response_list = []
# system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
# for index, row in final_df[final_df['EMPLOYMENT_adverse'] == True].iterrows():
#     free_text = row['text']
#     user_message = step1_query_optimized.format(free_text=free_text)
#     openai_message = create_prompt(system_message, user_message)
#     response = send_message(openai_message, deployment_name)
    
#     index_list.append(index)
#     llm_response_list.append(response)
#     print(free_text)
#     print(response)
#     print()

# llm_employment_adverse_step1 = pd.DataFrame({'index': index_list, 'llm_employment_adverse': llm_response_list})

# with open('llm_employment_adverse_step1.pkl', 'wb') as file:
#     pickle.dump(llm_employment_adverse_step1, file)

In [22]:
final_df['RELATIONSHIP_nonadverse'].value_counts()

RELATIONSHIP_nonadverse
False    5157
True      157
Name: count, dtype: int64

In [23]:
# index_list = []
# llm_response_list = []
# system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
# for index, row in final_df[final_df['RELATIONSHIP_nonadverse'] == True].iterrows():
#     free_text = row['text']
#     user_message = step1_query_optimized.format(free_text=free_text)
#     openai_message = create_prompt(system_message, user_message)
#     response = send_message(openai_message, deployment_name)
    
#     index_list.append(index)
#     llm_response_list.append(response)
#     print(free_text)
#     print(response)
#     print()

# llm_relationship_nonadverse_step1 = pd.DataFrame({'index': index_list, 'llm_relationship_nonadverse': llm_response_list})

# with open('llm_relationship_nonadverse_step1.pkl', 'wb') as file:
#     pickle.dump(llm_relationship_nonadverse_step1, file)

In [24]:
final_df['RELATIONSHIP_adverse'].value_counts()

RELATIONSHIP_adverse
False    5291
True       23
Name: count, dtype: int64

In [25]:
# index_list = []
# llm_response_list = []
# system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
# for index, row in final_df[final_df['RELATIONSHIP_adverse'] == True].iterrows():
#     free_text = row['text']
#     user_message = step1_query_optimized.format(free_text=free_text)
#     openai_message = create_prompt(system_message, user_message)
#     response = send_message(openai_message, deployment_name)
    
#     index_list.append(index)
#     llm_response_list.append(response)
#     print(free_text)
#     print(response)
#     print()

# llm_relationship_adverse_step1 = pd.DataFrame({'index': index_list, 'llm_relationship_adverse': llm_response_list})

# with open('llm_relationship_adverse_step1.pkl', 'wb') as file:
#     pickle.dump(llm_relationship_adverse_step1, file)

In [26]:
final_df['TRANSPORTATION_distance'].value_counts()

TRANSPORTATION_distance
0.0    5319
1.0       2
Name: count, dtype: int64

In [27]:
# index_list = []
# llm_response_list = []
# system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
# for index, row in final_df[final_df['TRANSPORTATION_distance'] == True].iterrows():
#     free_text = row['text']
#     user_message = step1_query_optimized.format(free_text=free_text)
#     openai_message = create_prompt(system_message, user_message)
#     response = send_message(openai_message, deployment_name)
    
#     index_list.append(index)
#     llm_response_list.append(response)
#     print(free_text)
#     print(response)
#     print()

# llm_transportation_distance_step1 = pd.DataFrame({'index': index_list, 'llm_transportation_distance': llm_response_list})

# with open('llm_transportation_distance_step1.pkl', 'wb') as file:
#     pickle.dump(llm_transportation_distance_step1, file)

In [28]:
final_df['TRANSPORTATION_resource'].value_counts()

TRANSPORTATION_resource
0.0    5321
Name: count, dtype: int64

In [29]:
final_df['TRANSPORTATION_other'].value_counts()

TRANSPORTATION_other
0.0    5320
1.0       1
Name: count, dtype: int64

In [30]:
final_df['HOUSING_poor'].value_counts()

HOUSING_poor
0.0    5319
1.0       2
Name: count, dtype: int64

In [ ]:
# index_list = []
# llm_response_list = []
# system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
# for index, row in final_df[final_df['HOUSING_poor'] == True].iterrows():
#     free_text = row['text']
#     user_message = step1_query_optimized.format(free_text=free_text)
#     openai_message = create_prompt(system_message, user_message)
#     response = send_message(openai_message, deployment_name)
    
#     index_list.append(index)
#     llm_response_list.append(response)
#     print(free_text)
#     print(response)
#     print()

# llm_housing_poor_step1 = pd.DataFrame({'index': index_list, 'llm_housing_poor': llm_response_list})

# with open('llm_housing_poor_step1.pkl', 'wb') as file:
#     pickle.dump(llm_housing_poor_step1, file)

In [31]:
final_df['HOUSING_undomiciled'].value_counts()

HOUSING_undomiciled
0.0    5321
Name: count, dtype: int64

In [32]:
final_df['HOUSING_other'].value_counts()

HOUSING_other
0.0    5320
1.0       1
Name: count, dtype: int64

In [33]:
final_df['RELATIONSHIP_married'].value_counts()

RELATIONSHIP_married
0.0    5189
1.0     132
Name: count, dtype: int64

In [34]:
# index_list = []
# llm_response_list = []
# system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
# for index, row in final_df[final_df['RELATIONSHIP_married'] == True].iterrows():
#     free_text = row['text']
#     user_message = step1_query_optimized.format(free_text=free_text)
#     openai_message = create_prompt(system_message, user_message)
#     response = send_message(openai_message, deployment_name)
    
#     index_list.append(index)
#     llm_response_list.append(response)
#     print(free_text)
#     print(response)
#     print()

# llm_relationship_married_step1 = pd.DataFrame({'index': index_list, 'llm_relationship_married': llm_response_list})

# with open('llm_relationship_married_step1.pkl', 'wb') as file:
#     pickle.dump(llm_relationship_married_step1, file)

In [35]:
final_df['RELATIONSHIP_partnered'].value_counts()

RELATIONSHIP_partnered
0.0    5296
1.0      25
Name: count, dtype: int64

In [36]:
# index_list = []
# llm_response_list = []
# system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
# for index, row in final_df[final_df['RELATIONSHIP_partnered'] == True].iterrows():
#     free_text = row['text']
#     user_message = step1_query_optimized.format(free_text=free_text)
#     openai_message = create_prompt(system_message, user_message)
#     response = send_message(openai_message, deployment_name)
    
#     index_list.append(index)
#     llm_response_list.append(response)
#     print(free_text)
#     print(response)
#     print()

# llm_relationship_partnered_step1 = pd.DataFrame({'index': index_list, 'llm_relationship_partnered': llm_response_list})

# with open('llm_relationship_partnered_step1.pkl', 'wb') as file:
#     pickle.dump(llm_relationship_partnered_step1, file)

In [37]:
final_df['RELATIONSHIP_divorced'].value_counts()

RELATIONSHIP_divorced
0.0    5304
1.0      17
Name: count, dtype: int64

In [38]:
# index_list = []
# llm_response_list = []
# system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
# for index, row in final_df[final_df['RELATIONSHIP_divorced'] == True].iterrows():
#     free_text = row['text']
#     user_message = step1_query_optimized.format(free_text=free_text)
#     openai_message = create_prompt(system_message, user_message)
#     response = send_message(openai_message, deployment_name)
    
#     index_list.append(index)
#     llm_response_list.append(response)
#     print(free_text)
#     print(response)
#     print()

# llm_relationship_divorced_step1 = pd.DataFrame({'index': index_list, 'llm_relationship_divorced': llm_response_list})

# with open('llm_relationship_divorced_step1.pkl', 'wb') as file:
#     pickle.dump(llm_relationship_divorced_step1, file)

In [39]:
final_df['RELATIONSHIP_widowed'].value_counts()

RELATIONSHIP_widowed
0.0    5320
1.0       1
Name: count, dtype: int64

In [40]:
# index_list = []
# llm_response_list = []
# system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
# for index, row in final_df[final_df['RELATIONSHIP_widowed'] == True].iterrows():
#     free_text = row['text']
#     user_message = step1_query_optimized.format(free_text=free_text)
#     openai_message = create_prompt(system_message, user_message)
#     response = send_message(openai_message, deployment_name)
    
#     index_list.append(index)
#     llm_response_list.append(response)
#     print(free_text)
#     print(response)
#     print()

# llm_relationship_widowed_step1 = pd.DataFrame({'index': index_list, 'llm_relationship_widowed': llm_response_list})

# with open('llm_relationship_widowed_step1.pkl', 'wb') as file:
#     pickle.dump(llm_relationship_widowed_step1, file)

In [41]:
final_df['RELATIONSHIP_single'].value_counts()

RELATIONSHIP_single
0.0    5316
1.0       5
Name: count, dtype: int64

In [42]:
# index_list = []
# llm_response_list = []
# system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
# for index, row in final_df[final_df['RELATIONSHIP_single'] == True].iterrows():
#     free_text = row['text']
#     user_message = step1_query_optimized.format(free_text=free_text)
#     openai_message = create_prompt(system_message, user_message)
#     response = send_message(openai_message, deployment_name)
    
#     index_list.append(index)
#     llm_response_list.append(response)
#     print(free_text)
#     print(response)
#     print()

# llm_relationship_single_step1 = pd.DataFrame({'index': index_list, 'llm_relationship_single': llm_response_list})

# with open('llm_relationship_single_step1.pkl', 'wb') as file:
#     pickle.dump(llm_relationship_single_step1, file)

In [43]:
final_df['EMPLOYMENT_employed'].value_counts()

EMPLOYMENT_employed
0.0    5290
1.0      31
Name: count, dtype: int64

In [44]:
# index_list = []
# llm_response_list = []
# system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
# for index, row in final_df[final_df['EMPLOYMENT_employed'] == True].iterrows():
#     free_text = row['text']
#     user_message = step1_query_optimized.format(free_text=free_text)
#     openai_message = create_prompt(system_message, user_message)
#     response = send_message(openai_message, deployment_name)
    
#     index_list.append(index)
#     llm_response_list.append(response)
#     print(free_text)
#     print(response)
#     print()

# llm_employment_employed_step1 = pd.DataFrame({'index': index_list, 'llm_employment_employed': llm_response_list})

# with open('llm_employment_employed_step1.pkl', 'wb') as file:
#     pickle.dump(llm_employment_employed_step1, file)

In [45]:
final_df['EMPLOYMENT_underemployed'].value_counts()

EMPLOYMENT_underemployed
0.0    5318
1.0       3
Name: count, dtype: int64

In [46]:
# index_list = []
# llm_response_list = []
# system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
# for index, row in final_df[final_df['EMPLOYMENT_underemployed'] == True].iterrows():
#     free_text = row['text']
#     user_message = step1_query_optimized.format(free_text=free_text)
#     openai_message = create_prompt(system_message, user_message)
#     response = send_message(openai_message, deployment_name)
    
#     index_list.append(index)
#     llm_response_list.append(response)
#     print(free_text)
#     print(response)
#     print()

# llm_employment_underemployed_step1 = pd.DataFrame({'index': index_list, 'llm_employment_underemployed': llm_response_list})

# with open('llm_employment_underemployed_step1.pkl', 'wb') as file:
#     pickle.dump(llm_employment_underemployed_step1, file)

In [47]:
final_df['EMPLOYMENT_unemployed'].value_counts()

EMPLOYMENT_unemployed
0.0    5311
1.0      10
Name: count, dtype: int64

In [48]:
# index_list = []
# llm_response_list = []
# system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
# for index, row in final_df[final_df['EMPLOYMENT_unemployed'] == True].iterrows():
#     free_text = row['text']
#     user_message = step1_query_optimized.format(free_text=free_text)
#     openai_message = create_prompt(system_message, user_message)
#     response = send_message(openai_message, deployment_name)
    
#     index_list.append(index)
#     llm_response_list.append(response)
#     print(free_text)
#     print(response)
#     print()

# llm_employment_unemployed_step1 = pd.DataFrame({'index': index_list, 'llm_employment_unemployed': llm_response_list})

# with open('llm_employment_unemployed_step1.pkl', 'wb') as file:
#     pickle.dump(llm_employment_unemployed_step1, file)

In [49]:
final_df['EMPLOYMENT_disability'].value_counts()

EMPLOYMENT_disability
0.0    5311
1.0      10
Name: count, dtype: int64

In [50]:
# index_list = []
# llm_response_list = []
# system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
# for index, row in final_df[final_df['EMPLOYMENT_disability'] == True].iterrows():
#     free_text = row['text']
#     user_message = step1_query_optimized.format(free_text=free_text)
#     openai_message = create_prompt(system_message, user_message)
#     response = send_message(openai_message, deployment_name)
    
#     index_list.append(index)
#     llm_response_list.append(response)
#     print(free_text)
#     print(response)
#     print()

# llm_employment_disability_step1 = pd.DataFrame({'index': index_list, 'llm_employment_disability': llm_response_list})

# with open('llm_employment_disability_step1.pkl', 'wb') as file:
#     pickle.dump(llm_employment_disability_step1, file)

In [51]:
final_df['EMPLOYMENT_retired'].value_counts()

EMPLOYMENT_retired
0.0    5316
1.0       5
Name: count, dtype: int64

In [52]:
# index_list = []
# llm_response_list = []
# system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
# for index, row in final_df[final_df['EMPLOYMENT_retired'] == True].iterrows():
#     free_text = row['text']
#     user_message = step1_query_optimized.format(free_text=free_text)
#     openai_message = create_prompt(system_message, user_message)
#     response = send_message(openai_message, deployment_name)
    
#     index_list.append(index)
#     llm_response_list.append(response)
#     print(free_text)
#     print(response)
#     print()

# llm_employment_retired_step1 = pd.DataFrame({'index': index_list, 'llm_employment_retired': llm_response_list})

# with open('llm_employment_retired_step1.pkl', 'wb') as file:
#     pickle.dump(llm_employment_retired_step1, file)

In [53]:
final_df['EMPLOYMENT_student'].value_counts()

EMPLOYMENT_student
0.0    5310
1.0      11
Name: count, dtype: int64

In [54]:
# index_list = []
# llm_response_list = []
# system_message = "You are an information extract tool that follows instructions very well and is specifically trained to extract social determinants of health elements from hospital generated free-text."
# for index, row in final_df[final_df['EMPLOYMENT_student'] == True].iterrows():
#     free_text = row['text']
#     user_message = step1_query_optimized.format(free_text=free_text)
#     openai_message = create_prompt(system_message, user_message)
#     response = send_message(openai_message, deployment_name)
    
#     index_list.append(index)
#     llm_response_list.append(response)
#     print(free_text)
#     print(response)
#     print()

# llm_employment_student_step1 = pd.DataFrame({'index': index_list, 'llm_employment_student': llm_response_list})

# with open('llm_employment_student_step1.pkl', 'wb') as file:
#     pickle.dump(llm_employment_student_step1, file)

### Extracting Insights

In [55]:
recall_insights = {}

In [56]:
with open('llm_employment_nonadverse_step1.pkl', 'rb') as file:
    llm_employment_nonadverse_step1 = pickle.load(file)

recall_insights['llm_employment_nonadverse_step1'] = sum(llm_employment_nonadverse_step1['llm_employment_nonadverse'] == 'YES')/len(llm_employment_nonadverse_step1)

In [57]:
with open('llm_employment_adverse_step1.pkl', 'rb') as file:
    llm_employment_adverse_step1 = pickle.load(file)
    
recall_insights['llm_employment_adverse_step1'] = sum(llm_employment_adverse_step1['llm_employment_adverse'] == 'YES')/len(llm_employment_adverse_step1)

In [58]:
with open('llm_employment_employed_step1.pkl', 'rb') as file:
    llm_employment_employed_step1 = pickle.load(file)
    
recall_insights['llm_employment_employed_step1'] = sum(llm_employment_employed_step1['llm_employment_employed'] == 'YES')/len(llm_employment_employed_step1)

In [59]:
with open('llm_employment_retired_step1.pkl', 'rb') as file:
    llm_employment_retired_step1 = pickle.load(file)
    
recall_insights['llm_employment_retired_step1'] = sum(llm_employment_retired_step1['llm_employment_retired'] == 'YES')/len(llm_employment_retired_step1)

In [60]:
with open('llm_employment_student_step1.pkl', 'rb') as file:
    llm_employment_student_step1 = pickle.load(file)
    
recall_insights['llm_employment_student_step1'] = sum(llm_employment_student_step1['llm_employment_student'] == 'YES')/len(llm_employment_student_step1)

In [61]:
with open('llm_employment_underemployed_step1.pkl', 'rb') as file:
    llm_employment_underemployed_step1 = pickle.load(file)
    
recall_insights['llm_employment_underemployed_step1'] = sum(llm_employment_underemployed_step1['llm_employment_underemployed'] == 'YES')/len(llm_employment_underemployed_step1)

In [62]:
with open('llm_employment_unemployed_step1.pkl', 'rb') as file:
    llm_employment_unemployed_step1 = pickle.load(file)
    
recall_insights['llm_employment_unemployed_step1'] = sum(llm_employment_unemployed_step1['llm_employment_unemployed'] == 'YES')/len(llm_employment_unemployed_step1)

In [63]:
with open('llm_housing_poor_step1.pkl', 'rb') as file:
    llm_housing_poor_step1 = pickle.load(file)
    
recall_insights['llm_housing_poor_step1'] = sum(llm_housing_poor_step1['llm_housing_poor'] == 'YES')/len(llm_housing_poor_step1)

In [64]:
with open('llm_relationship_adverse_step1.pkl', 'rb') as file:
    llm_relationship_adverse_step1 = pickle.load(file)
    
recall_insights['llm_relationship_adverse_step1'] = sum(llm_relationship_adverse_step1['llm_relationship_adverse'] == 'YES')/len(llm_relationship_adverse_step1)

In [65]:
with open('llm_relationship_divorced_step1.pkl', 'rb') as file:
    llm_relationship_divorced_step1 = pickle.load(file)
    
recall_insights['llm_relationship_divorced_step1'] = sum(llm_relationship_divorced_step1['llm_relationship_divorced'] == 'YES')/len(llm_relationship_divorced_step1)

In [66]:
with open('llm_relationship_married_step1.pkl', 'rb') as file:
    llm_relationship_married_step1 = pickle.load(file)
    
recall_insights['llm_relationship_married_step1'] = sum(llm_relationship_married_step1['llm_relationship_married'] == 'YES')/len(llm_relationship_married_step1)

In [67]:
with open('llm_relationship_nonadverse_step1.pkl', 'rb') as file:
    llm_relationship_nonadverse_step1 = pickle.load(file)
    
recall_insights['llm_relationship_nonadverse_step1'] = sum(llm_relationship_nonadverse_step1['llm_relationship_nonadverse'] == 'YES')/len(llm_relationship_nonadverse_step1)

In [68]:
with open('llm_relationship_partnered_step1.pkl', 'rb') as file:
    llm_relationship_partnered_step1 = pickle.load(file)
    
recall_insights['llm_relationship_partnered_step1'] = sum(llm_relationship_partnered_step1['llm_relationship_partnered'] == 'YES')/len(llm_relationship_partnered_step1)

In [69]:
with open('llm_relationship_single_step1.pkl', 'rb') as file:
    llm_relationship_single_step1 = pickle.load(file)
    
recall_insights['llm_relationship_single_step1'] = sum(llm_relationship_single_step1['llm_relationship_single'] == 'YES')/len(llm_relationship_single_step1)

In [70]:
with open('llm_relationship_widowed_step1.pkl', 'rb') as file:
    llm_relationship_widowed_step1 = pickle.load(file)
    
recall_insights['llm_relationship_widowed_step1'] = sum(llm_relationship_widowed_step1['llm_relationship_widowed'] == 'YES')/len(llm_relationship_widowed_step1)

In [71]:
with open('llm_transportation_distance_step1.pkl', 'rb') as file:
    llm_transportation_distance_step1 = pickle.load(file)
    
recall_insights['llm_transportation_distance_step1'] = sum(llm_transportation_distance_step1['llm_transportation_distance'] == 'YES')/len(llm_transportation_distance_step1)

In [72]:
recall_insights

{'llm_employment_nonadverse_step1': 0.9777777777777777,
 'llm_employment_adverse_step1': 0.9545454545454546,
 'llm_employment_employed_step1': 0.967741935483871,
 'llm_employment_retired_step1': 1.0,
 'llm_employment_student_step1': 1.0,
 'llm_employment_underemployed_step1': 1.0,
 'llm_employment_unemployed_step1': 1.0,
 'llm_housing_poor_step1': 1.0,
 'llm_relationship_adverse_step1': 0.30434782608695654,
 'llm_relationship_divorced_step1': 0.7058823529411765,
 'llm_relationship_married_step1': 0.5681818181818182,
 'llm_relationship_nonadverse_step1': 0.28662420382165604,
 'llm_relationship_partnered_step1': 0.6,
 'llm_relationship_single_step1': 0.4,
 'llm_relationship_widowed_step1': 1.0,
 'llm_transportation_distance_step1': 1.0}